In [ ]:
import librosa
import librosa.display
import numpy as np
import madmom
import matplotlib.pyplot as plt
import mir_eval
import pandas as pd

import IPython.display as ipd

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
import utils

In [ ]:
# experimentos baseados no artigo 
# Evaluation Methods for Musical Audio Beat Tracking

In [ ]:
FS = 44100

In [ ]:
candombe_audio_path = '../datasets/candombe/csic.1995_ansina2_04.wav'

In [ ]:
candombe, _ = librosa.load(candombe_audio_path, sr=FS)

In [ ]:
ipd.Audio(candombe, rate=FS)

In [ ]:
candombe_stft = np.abs(librosa.stft(candombe[0:10*FS]))

In [ ]:
librosa.display.specshow(librosa.amplitude_to_db(candombe_stft, ref=np.max), y_axis='log', x_axis='time')

In [ ]:
bpm, beat_frame = librosa.beat.beat_track(candombe, FS)
librosa_timestamps = librosa.frames_to_time(beat_frame, FS)

In [ ]:
madmom_beat_processor = madmom.features.downbeats.RNNDownBeatProcessor(num_threads=4)
madmom_beat_decoder = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=100)
madmom_track = madmom_beat_decoder(madmom_beat_processor(candombe_audio_path))
madmom_timestamps, madmom_beats = madmom_track[:, 0], madmom_track[:, 1]

In [ ]:
x_df = pd.read_csv(candombe_audio_path.replace('.wav', '.csv'), names=["timestamp", "beat"])
ground_truth = x_df['timestamp'].values

In [ ]:
# TODO: testar plots mais interativos
utils.plot_comparison(candombe, FS, ground_truth, librosa_timestamps, 0, 10)

In [ ]:
utils.plot_comparison(candombe, FS, ground_truth, librosa_timestamps, 70, 80)

In [ ]:
utils.plot_comparison(candombe, FS, ground_truth, candombe_timestamps, 0, 10)

In [ ]:
utils.plot_comparison(candombe, FS, ground_truth, madmom_timestamps, 70, 80)

In [ ]:
utils.plot_comparison(candombe, FS, ground_truth, madmom_timestamps, 70, 80)

In [ ]:
np.diff(librosa_timestamps).mean()

In [ ]:
np.diff(madmom_timestamps).mean()

In [ ]:
np.diff(ground_truth).mean()

In [ ]:
plt.subplot(3,1,1)
plt.title('Intervalos entre beats (IBI) - librosa')
plt.hist(np.diff(librosa_timestamps), bins=50, range=(0.2,0.7), label="librosa")
plt.legend()
plt.subplot(3,1,2)
plt.title('Intervalos entre beats (IBI) - madmom')
plt.hist(np.diff(madmom_timestamps), bins=50, range=(0.2,0.7), label="madmom")
plt.legend()
plt.subplot(3,1,3)
plt.title('Intervalos entre anotações (IAI)')
plt.hist(np.diff(ground_truth), bins=50, range=(0.2,0.7), label="ground_truth")
plt.xlabel('Beat Length (seconds)')
plt.ylabel('Count')
plt.legend()

In [ ]:
wrong_click_sound = np.sin(2*np.pi*np.arange(FS*.1)*500/(1.*FS))
wrong_click_sound *= np.exp(-np.arange(FS*.1)/(FS*.01)) # exponential decay

correct_clicks = mir_eval.sonify.clicks(ground_truth, FS, click=None, length=len(x))
wrong_clicks = mir_eval.sonify.clicks(candombe_timestamps, FS, click=wrong_click_sound, length=len(x))

In [ ]:
click_excerpt = candombe[60*FS:70*FS] + correct_clicks[60*FS:70*FS] + wrong_clicks[60*FS:70*FS] #
ipd.Audio(click_excerpt, rate=FS)

# Notação 
* B = sequência de estimações
* $\gamma_b$ = timestamp do b-ésimo beat
* J = sequência de valores de referência
* $a_j$ = valor de referência da j-ésima anotação 
* Intervalo entre beats (Inner Beat Interval ou IBI) = $\Delta_b = \gamma_b - \gamma_{b-1}$
* Intervalo entre anotações (inner-annotation-interval ou IAI) = $\Delta_j = a_j - a_{j-1}$

# F-measure
Também é conhecido como F-score. Leva em consideração a precisão 

$$
p = \frac{c}{c + f^+}
$$

e o recall (proporção de beats que estão corretos)

$$
r = \frac{c}{c + f^-}
$$

e então 

$$
F = \frac{2pr}{p+r} = \frac{2c}{2c + f^+ + f^-}
$$

Ou seja, se um beat estiver no contratempo, o valor da métrica será $0$. 

In [ ]:
mir_eval.beat.f_measure(ground_truth, librosa_timestamps)

In [ ]:
mir_eval.beat.f_measure(ground_truth, madmom_timestamps)

# Cemgil et al
Essa métrica usa uma função de erro gaussiana W que penaliza a acurácia de uma estimativa de acordo com a sua distância em relação ao valor de referência. Funciona como uma janela de tolerância 

$$
W(x) = \exp(-x^2/2\sigma_e^2)
$$

onde $x = \gamma_b - a_j$ e o desvio padrão é definido como 
$\sigma_e=40ms$. 

In [ ]:
# retorna cemgil_score, cemgil_max
mir_eval.beat.cemgil(ground_truth, librosa_timestamps)

In [ ]:
mir_eval.beat.cemgil(ground_truth, madmom_timestamps)

# PScore

* $T_a(n)$ = trem de pulsos para valores de referência (ground_truth)
* $T_{\gamma}(n)$ = trem de pulsos para valores estimados
$$
\begin{equation}
    T_a(n) =
    \begin{cases}
      0, & \text{if}\ n=a_j \\
      1, & \text{caso contrário}
    \end{cases}
  \end{equation}
$$

$$
\begin{equation}
    T_{\gamma}(n) =
    \begin{cases}
      0, & \text{if}\ n=\gamma_b \\
      1, & \text{caso contrário}
    \end{cases}
  \end{equation}
$$

As medidas são feitas dentro de uma janela $w$, que tem um valor 
empírico de 20% da mediana dos intervalos entre-anotações 
($\Delta_j$).  Ou seja $w = 0.2\cdot\;median(\Delta_j)$. O resultado da correlação é normalizado pelo máximo entre o total de anotações e o total de estimativas, evitando assim casos em que $T_{\gamma}$ fosse uma função uniforme e sua correlação com $T_a$ fosse maximizada.

$$ 
PScore = \frac{\sum_w T_a *_{(w)}T_{\gamma}}{\max(J, B)}
$$

In [ ]:
mir_eval.beat.p_score(ground_truth, librosa_timestamps)

In [ ]:
mir_eval.beat.p_score(ground_truth, madmom_timestamps)

# Goto e Muraoka

Essa métrica avalia as medições de beat como "corretas" ou "incorretas". O resultado é binário e vale 1 se seguir uma série de 
critérios heurísticos.

Nesse caso, pra cada valor referência $a_j$ calculamos o erro 
$\zeta_j$ em relação a estimativa $\gamma_b$ mais próxima e ao 
intervalo entre beats $\Delta_j$ mais próximo.

Os critérios heurísticos tratam da média e desvio padrão dos erros
$\zeta_k$ e da proximidade das estimativas $\gamma_b$.

In [ ]:
mir_eval.beat.goto(ground_truth, librosa_timestamps)

In [ ]:
mir_eval.beat.goto(ground_truth, librosa_timestamps + shift)

# Avaliação baseada em Continuidade

Esse tipo de avaliação segue o mesmo conceito de continuidade que é 
apresentado na medida de Goto e  Muraoka. A ideia é notar se as 
estimativas consistentemente estão dentro da janela de tolerância $\theta$.
Ou seja, um beat $\gamma_b$ só vai ser considerado correto se ele e 
seu antecessor, $\gamma_{b-1}$ estão dentro de suas respectivas 
janelas. 

* (i) $a_j - \theta\Delta_j < \gamma_b < a_j + \theta\Delta_j$
* (ii) $a_{j-1} - \theta\Delta_{j-1} < \gamma_{b-1} < a_{j-1} + \theta\Delta_{j-1}$
* (iii) $(1-\theta)\Delta_j < \Delta_b < (1+\theta)\Delta_j$

Ao compararmos cada beat $\gamma_b$ a cada anotação $a_j$ considerando as condições citadas acima podemos encontrar o número de beats corretos em cada segmento $\Upsilon_m$. E a partir disso, podemos calcular a razão do maior segmento com beats corretos em relação ao tamanho total da entrada.

$CML_c$ = Correct Metrical Level. Requer continuidade. 
$$
CML_c = \frac{\max(\Upsilon_m)}{J}
$$

Essa métrica reflete apenas o maior segmento e é portanto cega a 
outros beats que também foram corretos e satisfizeram as condições (i) - (iii). 
Se um beat incorreto acontece e esse erro está no meio da 
entrada, isso resultaria em um $CML_c = 50\%$.

Para considerar outros beats corretos fora maior segmento 
$\Upsilon_m$, existe a $CML_t$, que considera o total de beats corretos em todos os segmentos.

$CML_t$ = Correct Metrical Level. Não requer continuidade. 
$$
CML_t = \frac{\sum^M_{m=1}\Upsilon_m}{J}
$$

Para considerar andamentos diferentes, existe a AMLt e AMLc. Se, por exemplo, as marcações tiverem o dobro do BPM ou estiverem marcadas no offbeat, essas métricas ainda assim dão valores altos.

In [ ]:
# librosa
CMLc, CMLt, AMLc, AMLt = mir_eval.beat.continuity(ground_truth, librosa_timestamps)
print(f"CMLc = {CMLc}\nCMLt = {CMLt}\nAMLc = {AMLc}\nAMLt = {AMLt}")

In [ ]:
# madmom
CMLc, CMLt, AMLc, AMLt = mir_eval.beat.continuity(ground_truth, madmom_timestamps)
print(f"CMLc = {CMLc}\nCMLt = {CMLt}\nAMLc = {AMLc}\nAMLt = {AMLt}")

# Janelas de tolerância

As janelas de tolerância podem ser um problema. Deixar o valor delas muito baixo pode fazer com que mesmo beats corretos com algum delay inerente à tarefa não sejam classificados como tal. Da mesma forma, aumentar muito o valor da janela pode fazer com que beats errados sejam classificados como correto.

In [ ]:
mir_eval.beat.continuity(
    ground_truth,
    librosa_timestamps,
    continuity_phase_threshold = 0.3,
    continuity_period_threshold = 0.3)

In [ ]:
mir_eval.beat.continuity(
    ground_truth,
    madmom_timestamps,
    continuity_phase_threshold = 0.3,
    continuity_period_threshold = 0.3)

# Deslocamento

In [ ]:
halfbeat_madmom = np.diff(madmom_timestamps).mean() / 4
halfbeat_librosa = np.diff(librosa_timestamps).mean() / 4

In [ ]:
mir_eval.beat.f_measure(ground_truth, librosa_timestamps + halfbeat_librosa)

In [ ]:
mir_eval.beat.f_measure(ground_truth, madmom_timestamps + halfbeat_madmom)

# Perguntas
- por que há esse deslocamento? a informação musical está mais próxima do offbeat? <- calcular o IBI pra descobrir qual é o valor em segundos/milissegundos do offbeat
- qual assumpção que funciona pra músicas ocidentais não funciona pra esse gênero? por quê?
- o deslocamento acontece com todos os métodos de detecção usados?

# Próximos passos
- entender e adicionar o método de information gain
- ajustar função de plot pra mostrar os onsets em vez do sinal todo
- para os testes com todo o dataset: implementar um histograma de beat pra conseguir comparar as estimativas e os valores de referência


# Referências
- [Evaluation Methods for Musical Audio BeatTracking Algorithms]()
- [METHODOLOGY AND TOOLS FOR THE EVALUATION OF AUTOMATIC ONSET DETECTION ALGORITHMS IN MUSIC](https://www.ee.columbia.edu/~dpwe/ismir2004/CRFILES/paper188.pdf)